# Jupyter notebook based on ImageD11 to process 3DXRD data
# Written by Haixing Fang, Jon Wright and James Ball
## Date: 10/09/2024

This notebook will help you to extract the locations of diffraction peaks on your detector images.

It will also merge together your 2D spots (on a stack of detector images with different omega angles).

We merge across omega because we often see the same spot twice on multiple detector images.

The results are saved to the PROCESSED_DATA folder of the experiment, inside the sample and dataset folders that you select within this notebook

## NOTE: These notebooks are under active development
They require the latest version of ImageD11 from Git to run.

If you don't have this set up yet, you can run the below cell.

It will automatically download and install ImageD11 to your home directory

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

In [ ]:
# import functions we need

import glob, pprint

import ImageD11.sinograms.dataset
import ImageD11.sinograms.lima_segmenter
import ImageD11.sinograms.assemble_label
import ImageD11.sinograms.properties

import numpy as np
import fabio
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from skimage import filters, measure, morphology
import ipywidgets as widgets
import h5py
from IPython.display import display
%matplotlib widget

from ImageD11.nbGui import nb_utils as utils
from ImageD11.nbGui import segmenter_gui

from ImageD11.frelon_peaksearch import worker, segment_dataset, guess_bg


# Experts : update these files for your detector if you need to

# give dx/dy as tuple instead of spline
# Since 2024: there is no good spline for a detector at ID11.
# splinefile = ('/data/id11/3dxrd/inhouse/Frelon36/frelon36_spline_20240604_dx.edf','/data/id11/3dxrd/inhouse/Frelon36/frelon36_spline_20240604_dy.edf')
splinefile = ('/data/id11/3dxrd/inhouse/Frelon36/frelon36_spline_20240604_ftm1x1_dx.edf','/data/id11/3dxrd/inhouse/Frelon36/frelon36_spline_20240604_ftm1x1_dy.edf')
bgfile = None     #  'bg.edf'
maskfile = '/data/visitor/xa8/id11/20240917/PROCESSED_DATA/setup/ftm_mask.edf' # '/data/id11/inhouse1/ewoks/detectors/files/Frelon2k_C36/mask.edf'

detector = "frelon6"  # fixme - guess this from masterfile + scan
omegamotor = "frot"
dtymotor = "diffy"


#Define the initial parameters
options = {
    "bgfile":bgfile,
    "maskfile":maskfile,
    "threshold":500,
    "smoothsigma":1.0,
    "bgc":0.9,
    "minpx":3,
    "m_offset_thresh":100,
    "m_ratio_thresh":150,
}

In [ ]:
# Set up the file paths. Edit this if you are not at ESRF or not using the latest data policy.
dataroot, analysisroot = segmenter_gui.guess_ESRF_paths() 

if len(dataroot)==0:
    print("Please fix in the dataroot and analysisroot folder names above!!")
print('dataroot =',repr(dataroot))
print('analysisroot =',repr(analysisroot))

In [ ]:
# List the samples available:
segmenter_gui.printsamples(dataroot)

In [ ]:
# USER: Decide which sample
sample = 'S01_Pure_Al_After_PDF'

In [ ]:
# List the datasets for that sample:
segmenter_gui.printdatasets( dataroot, sample )

In [ ]:
# USER: Decide which dataset
dataset = "3dxrd_rt_04"

In [ ]:
# create ImageD11 dataset object

ds = ImageD11.sinograms.dataset.DataSet(dataroot=dataroot,
                                        analysisroot=analysisroot,
                                        sample=sample,
                                        dset=dataset,
                                        detector=detector,
                                        omegamotor=omegamotor,
                                        dtymotor=dtymotor)
ds.import_all(scans=["1.1"])
if len(splinefile) == 1:
    ds.splinefile = splinefile
else:
    ds.e2dxfile, ds.e2dyfile = splinefile
ds.maskfile = maskfile
ds.bgfile = bgfile
ds.save()

In [ ]:
bg = guess_bg( ds )
plt.imshow(bg)
fabio.edfimage.edfimage(bg).save('bg.edf')
plt.colorbar()

In [ ]:
%matplotlib widget

In [ ]:
ui = segmenter_gui.FrelonSegmenterGui(ds, worker, segment_dataset, **options)

In [ ]:
options = ui.getopts()
print(options)

In [ ]:
# now we run the segmenter on all our data

cf_2d, cf_3d = segment_dataset(ds, options)

In [ ]:
# display some peaks
f,a=plt.subplots(1,2,figsize=(12,6))
a[0].plot(cf_3d.f_raw,cf_3d.s_raw,'.',ms=1)
a[0].set(xlabel='fast index', ylabel='slow index',aspect='equal', title='peaks on detector')
a[1].plot(cf_3d.omega,cf_3d.sum_intensity,'.',ms=1)
a[1].set(xlabel='omega',ylabel='sum intensity',yscale='log',title='peaks vs omega');

In [ ]:
# frequent problem here: we do not have parameters at this point in the project. This comes later.
# parfile = os.path.join(ds.analysisroot, 'pars_tdxrd.json')
parfile = os.path.join(ds.analysisroot, 'Al.par')
ds.parfile = parfile

In [ ]:
ds.update_colfile_pars(cf_2d)  # phase is not needed as unitcell is not used in here
ds.update_colfile_pars(cf_3d)

In [ ]:
ImageD11.columnfile.colfile_to_hdf(cf_2d, ds.col2dfile)
ImageD11.columnfile.colfile_to_hdf(cf_3d, ds.col3dfile)

In [ ]:
ds.col2dfile

In [ ]:
ds.save()

In [ ]:
# change to 0 to allow all cells to be run automatically
if 1:
    raise ValueError("Hello!")

In [ ]:
# Now that we are happy with our segmentation parameters, we can run the below cell to do this in bulk for many samples/datasets
# by default this will do all samples in sample_list, all datasets with a prefix of dset_prefix
# you can add samples and datasets to skip in skips_dict

skips_dict = {
    "FeAu_0p5_tR": []
}

# USER: Change this prefix to match the naming convention of your datasets
# e.g if your slices are formatted like ["ff_z0", "ff_z1", "ff_z2"] etc, then the prefix could be "ff" or "ff_z"

dset_prefix = "ff"

sample_list = ["FeAu_0p5_tR"]
    
samples_dict = utils.find_datasets_to_process(ds.dataroot, skips_dict, dset_prefix, sample_list)
print(samples_dict)

# manual override example:
# samples_dict = {"FeAu_0p5_tR": ["ff1", "ff2"]}

In [ ]:
for sample, datasets in samples_dict.items():
    for dataset in datasets:
        print(f"Processing dataset {dataset} in sample {sample}")
        print("Importing DataSet object")
        ds = ImageD11.sinograms.dataset.DataSet(dataroot=ds.dataroot,
                                            analysisroot=ds.analysisroot,
                                            sample=sample,
                                            dset=dataset,
                                            detector=detector,
                                            omegamotor=omegamotor,
                                            dtymotor=dtymotor)
        
        if os.path.exists(ds.col2dfile):
            print(f"Found existing cf_2d for {dataset} in {sample}, skipping")
            continue
        
        ds.import_all(scans=["1.1"])
        print(f"I have a DataSet {ds.dset} in sample {ds.sample}")
        if len(splinefile) == 1:
            ds.splinefile = splinefile
        else:
            ds.e2dxfile, ds.e2dyfile = splinefile
        
        ds.splinefile = splinefile
        ds.maskfile = maskfile
        ds.bgfile = bgfile
        ds.parfile = parfile
        ds.save()
        
        print("Peaksearching")
        cf_2d, cf_3d = segment_dataset(ds, options)
        
        ds.update_colfile_pars( cf_2d )
        ds.update_colfile_pars( cf_3d )

        print("Saving peaks to file")
        ImageD11.columnfile.colfile_to_hdf(cf_2d, ds.col2dfile)
        ImageD11.columnfile.colfile_to_hdf(cf_3d, ds.col3dfile)
        
        ds.save()
print('Done!')